In [2]:
import re
import csv
import time
import nltk
import json
import string
import numpy as np
import pandas as pd
from nltk import stem
import sklearn.metrics
from random import randint
from numpy.linalg import norm
from nltk.corpus import stopwords
from sklearn import random_projection
from nltk.stem import WordNetLemmatizer


with open('../new_userDict.json', 'r') as fp:
    
    new_userDict = json.load(fp)
    
    
with open('../new_isbnDict.json', 'r') as fp:
    
    new_isbnDict = json.load(fp)
    
    
with open('../dict_row.json', 'r') as fp:
    
    dict_row = json.load(fp)
    
    
with open('../dict_col.json', 'r') as fp:
    
    dict_col = json.load(fp)
    
        
print("Ok")

Ok


In [2]:
with open('file/clusters2_new_userDict.json', 'r') as fp:
    
    clusters_new_userDict = json.load(fp)
    
with open('file/clusters2_new_isbnDict.json', 'r') as fp:
    
    clusters_new_isbnDict = json.load(fp)
    
    
with open('file/clusters_dict_row.json', 'r') as fp:
    
    clusters_dict_row = json.load(fp)
    
    
with open('file/clusters_dict_col.json', 'r') as fp:
    
    clusters_dict_col = json.load(fp)
    
    
print("Ok")

Ok


# Clustering on items (using utility matrix information)

In [3]:
def createSampleDict(new_isbnDict, new_userDict, t1, t2):
    

    small_isbnDict = {}

    for book in new_isbnDict:

        temp = new_isbnDict[book]
        i = 0

        for t in temp.values():
            if t != "0":
                i = i+1

        if i >t1:

            small_isbnDict[book] = new_isbnDict[book]   

    small_userDict = {}

    for user in new_userDict:

        temp = new_userDict[user]
        i = 0

        for t in temp.values():
            if t != "0":
                i = i+1

        if i >t2:

            small_userDict[user] = new_userDict[user]
            
            
    return small_isbnDict, small_userDict


def computeMatrices(train_userDict,train_isbnDict,small_userDict,small_isbnDict, dict_row, dict_col):

    n = len(small_isbnDict)
    m = len(small_userDict)
    
    index = sorted(small_userDict.keys())
    columns = sorted(small_isbnDict.keys())

    dict_row = {k:v for v,k in enumerate(index)}
    dict_col = {k:v for v,k in enumerate(columns)}

    u = np.zeros((m,n)) 
    R = np.zeros((m,n))
    for user in train_userDict:
        for isbn in train_userDict[user]:
            try:
                u[dict_row[user]][dict_col[isbn]] = train_userDict[user][isbn]
                R[dict_row[user]][dict_col[isbn]] = 1
            except:
                continue

    for isbn in train_isbnDict:
        for user in train_isbnDict[isbn]:
            try:
                u[dict_row[user]][dict_col[isbn]] = train_isbnDict[isbn][user]
                R[dict_row[user]][dict_col[isbn]] = 1
            except:
                continue
                
    small_utility_DataFrame = pd.DataFrame(u, index = index, columns = columns)

    return u, R, small_utility_DataFrame

In [ ]:
#U, R, utility_DataFrame, dict_row, dict_col = computeMatrices(new_userDict,new_isbnDict,new_userDict,new_isbnDict, dict_row, dict_col)

# clusterization items

In [ ]:
transformer = random_projection.SparseRandomProjection(n_components=100)

X_new = transformer.fit_transform(U.T)

X_new = X_new.T

books = utility_DataFrame.columns

In [ ]:
X_new.shape

In [ ]:
def clusteringItems(X_new,books,t=1000):

    j = 0
    b = [[i] for i in books]

    while len(b) > t:

        x = X_new[:,j]

        y_length = norm(X_new, axis = 0)
        x_length = y_length[j]

        num = (X_new.T*x).sum(axis=1)
        den = y_length*x_length

        similarity = num/den
        similarity = np.nan_to_num(similarity)
        similarity[j] = 0

        b[j].extend(b[similarity.argmax()])

        b = np.delete(b, similarity.argmax())

        z_index = np.where(X_new[:,j] !=0)[0]
        z1_index = np.where(X_new[:,similarity.argmax()] !=0)[0]

        non_inter = set(z1_index) - set(z_index)

        inter = set(z1_index).intersection(set(z_index))

        X_new[list(non_inter),j] = X_new[list(non_inter),similarity.argmax()]

        X_new[list(inter),j] = (X_new[list(inter),similarity.argmax()] +  X_new[list(inter),j])/2

        X_new = np.delete(X_new, similarity.argmax(), axis = 1 )

        #print(j, similarity.argmax())

        if similarity.argmax() > j:

            j = j+1

        if j == len(b):

            j = 0

        if j%3000 == 0:

            time.sleep(30)
            print(j)
            
    return b

In [ ]:
#b = clusteringItems(X_new,books)

#  create a relative order for clusters

In [ ]:
b_dict = {str(k):v for k,v in enumerate(b)}

# save the position of every isbn inside the clusters

In [7]:
isbn_cluster_book = {k:"" for k in list(new_isbnDict.keys())}

for cluster in b_dict:
    
    for b_c in b_dict[cluster]:
        
        isbn_cluster_book[b_c] = cluster

In [8]:
isbn_cluster_book

{'0061012203': '486',
 '0380470845': '710',
 '0380712040': '4',
 '3499100592': '347',
 '0425090825': '927',
 '0671876821': '852',
 '0767915747': '92',
 '0449149986': '338',
 '0345464958': '905',
 '0385336764': '584',
 '0373790406': '846',
 '0345423097': '718',
 '0684804212': '693',
 '0758205481': '562',
 '038509423X': '111',
 '0380789876': '206',
 '0445200456': '194',
 '0679447393': '304',
 '0312154305': '519',
 '0440207290': '690',
 '0345335651': '134',
 '0380801523': '522',
 '080071816X': '702',
 '0066238501': '793',
 '0151006903': '21',
 '0380710838': '8',
 '0425126870': '813',
 '0061030651': '120',
 '0345356535': '877',
 '0684863340': '477',
 '0345440072': '923',
 '0684804441': '33',
 '0312924607': '479',
 '0312983115': '193',
 '1551666561': '794',
 '0811808424': '142',
 '8481302651': '699',
 '0451408586': '411',
 '0316058432': '869',
 '0449006417': '50',
 '0805425500': '358',
 '0061013439': '222',
 '0778320057': '75',
 '0380775271': '725',
 '0749301384': '402',
 '0440234697': '803

# clustering users

In [ ]:
transformer = random_projection.SparseRandomProjection(n_components=100)

X_new_users = transformer.fit_transform(U)

users = utility_DataFrame.index

In [ ]:
X_new_users.shape

In [ ]:
def clusteringUsers(X_new_users, users, t=1000):
    
    i = 0
    u = [[i] for i in users]

    while len(u) > t:

        x = X_new_users[i,:]

        y_length = norm(X_new_users, axis = 1)
        x_length = y_length[i]

        num = (X_new_users*x).sum(axis=1)
        den = y_length*x_length

        similarity = num/den
        similarity = np.nan_to_num(similarity)
        similarity[i] = 0

        u[i].extend(u[similarity.argmax()])

        u = np.delete(u, similarity.argmax())

        z_index = np.where(X_new_users[i,:] !=0)[0]
        z1_index = np.where(X_new_users[similarity.argmax(),:] !=0)[0]

        non_inter = set(z1_index) - set(z_index)

        inter = set(z1_index).intersection(set(z_index))

        X_new_users[i,list(non_inter)] = X_new_users[similarity.argmax(),list(non_inter)]

        X_new_users[i,list(inter)] = (X_new_users[similarity.argmax(),list(inter)] +  X_new_users[i,list(inter)])/2

        X_new_users = np.delete(X_new_users, similarity.argmax(), axis = 0 )

        #print(j, similarity.argmax())

        if similarity.argmax() > i:

            i = i+1

        if i == len(u):

            i = 0

        if i%3000 == 0:

            time.sleep(30)
            print("Ok")
            
    return u

In [ ]:
u = clusteringUsers(X_new_users,users)

In [ ]:
u_dict = {str(k):v for k,v in enumerate(u)}

In [10]:
with open('cluster_book_dict.json', 'r') as fp:
    
    cluster_book_dict = json.load(fp)
    
with open('cluster_users_dict.json', 'r') as fp:
    
    cluster_users_dict = json.load(fp)

In [12]:
cluster_book_dict

{'199': ['0375703861',
  '0552994944',
  '0156767503',
  '0553260669',
  '0486266907',
  '0440206324',
  '1582341281',
  '0553283545',
  '0486406482',
  '0553571591',
  '155166884X',
  '0446353914',
  '0316507415',
  '006001315X',
  '0451162331'],
 '166': ['0373196318',
  '1551665859',
  '0373483791',
  '1551665395',
  '0373484461',
  '0440234808',
  '1551669633',
  '0380771713',
  '1551669625'],
 '178': ['0373832826',
  '0553290851',
  '0446364266',
  '0451148029',
  '0553575619',
  '0061090808',
  '0140376410',
  '0671741209',
  '0786889098'],
 '689': ['0670865796',
  '0875420036',
  '0836270061',
  '0671549170',
  '0061008079',
  '044018293X',
  '068484477X',
  '0380400480',
  '0380750848',
  '0525941908',
  '0385495323',
  '0451455509',
  '0670855030',
  '0895773619'],
 '171': ['0373243790',
  '0671774662',
  '067091021X',
  '0812521501',
  '0345322401',
  '0553295284',
  '0525934960',
  '0307020673',
  '0395159938',
  '0399148701',
  '0385505914',
  '0786865644',
  '0743452860',
 

In [13]:
user_cluster_users = {k:"" for k in list(new_userDict.keys())}

for cluster in u_dict:
    
    for u_c in u_dict[cluster]:
        
        user_cluster_users[u_c] = cluster

In [14]:
user_cluster_users

{'26209': '368',
 '200674': '808',
 '95574': '463',
 '24452': '867',
 '126654': '976',
 '128362': '934',
 '1652': '844',
 '162519': '305',
 '269386': '698',
 '46654': '231',
 '222138': '664',
 '271354': '843',
 '155306': '872',
 '45122': '539',
 '200252': '90',
 '24440': '433',
 '85379': '314',
 '214851': '151',
 '237170': '416',
 '272273': '832',
 '266126': '764',
 '162095': '586',
 '123485': '844',
 '231488': '863',
 '220522': '716',
 '137082': '571',
 '17361': '190',
 '67199': '363',
 '142680': '685',
 '130271': '579',
 '97906': '836',
 '132786': '869',
 '244139': '236',
 '37666': '378',
 '9454': '685',
 '91882': '482',
 '114255': '607',
 '157935': '642',
 '259205': '455',
 '11855': '154',
 '214821': '695',
 '173053': '517',
 '236032': '961',
 '5674': '484',
 '195943': '628',
 '251541': '2',
 '248922': '680',
 '48215': '66',
 '153496': '828',
 '18716': '801',
 '47034': '454',
 '96448': '984',
 '70541': '70',
 '26624': '575',
 '131311': '362',
 '216675': '298',
 '76701': '981',
 '471

# create a new new_userDict considering the actual clusterization

# clusters_new_userDict 
# key ----->  user cluster 
# value -----> book rated by users inside the clusters

In [ ]:
clusters_new_userDict = {k:{} for k in cluster_users_dict}

In [ ]:
for key in cluster_users_dict:
    
    for user in cluster_users_dict[key]:
        
        for book in new_userDict[user]:
            
            if book not in clusters_new_userDict[key]:
                
                clusters_new_userDict[key][book] = []
            
            clusters_new_userDict[key][book].append(int(new_userDict[user][book]))

In [ ]:
clusters_new_isbnDict = {k:{} for k in cluster_book_dict}

In [ ]:
for key in cluster_users_dict:
    
    for book in cluster_book_dict[key]:
        
        for user in new_isbnDict[book]:
            
            if user not in clusters_new_isbnDict[key]:
                
                clusters_new_isbnDict[key][user] = []
            
            clusters_new_isbnDict[key][user].append(int(new_isbnDict[book][user]))

In [14]:
for key in clusters_new_isbnDict:
    
    for book in clusters_new_isbnDict[key]:
        
        clusters_new_isbnDict[key][book] = str(np.mean(clusters_new_isbnDict[key][book]))

In [15]:
for key in clusters_new_userDict:
    
    for user in clusters_new_userDict[key]:
        
        clusters_new_userDict[key][user] = str(np.mean(clusters_new_userDict[key][user]))

In [ ]:
len(clusters_new_isbnDict["2"])

In [ ]:
new_userDict["11676"]

In [ ]:
utility_DataFrame.loc["11676"][utility_DataFrame.loc["11676"] !=0]

In [ ]:
clusters_index = sorted(clusters_new_userDict.keys())
clusters_columns = sorted(clusters_new_isbnDict.keys())

clusters_dict_row = {k:v for v,k in enumerate(clusters_index)}
clusters_dict_col = {k:v for v,k in enumerate(clusters_columns)}

In [4]:
U_cluster, R_cluster, utility_DataFrame_cluster = computeMatrices(clusters_new_userDict,clusters_new_isbnDict,
                                                                  clusters_new_userDict,clusters_new_isbnDict, 
                                                                  clusters_dict_row, clusters_dict_col)